In [1]:
#Import Dependencies
import pandas as pd
import requests
import json
import time
import numpy as np

In [2]:
#Query API and Convert to DataFrame

#Read csv with proper encoding
path = "U.S. Released Movies_ 1972-2016.csv"
movies = pd.read_csv(path, encoding='latin1')
movies.head()

,Position,Const,Created,Modified,Description,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,1,tt0110912,4/23/2013,4/23/2013,NaN,Pulp Fiction,https://www.imdb.com/title/tt0110912/,movie,8.9,154.0,1994.0,"Crime, Drama",1607823.0,5/21/1994,Quentin Tarantino
1,2,tt1872181,4/23/2013,4/23/2013,NaN,The Amazing Spider-Man 2,https://www.imdb.com/title/tt1872181/,movie,6.6,142.0,2014.0,"Action, Adventure, Sci-Fi",381550.0,4/10/2014,Marc Webb
2,3,tt0111161,4/23/2013,4/23/2013,NaN,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,movie,9.3,142.0,1994.0,Drama,2057262.0,9/10/1994,Frank Darabont
3,4,tt0076759,4/23/2013,4/23/2013,NaN,Star Wars,https://www.imdb.com/title/tt0076759/,movie,8.6,121.0,1977.0,"Action, Adventure, Fantasy, Sci-Fi",1102354.0,5/25/1977,George Lucas
4,5,tt0088763,4/23/2013,4/23/2013,NaN,Back to the Future,https://www.imdb.com/title/tt0088763/,movie,8.5,116.0,1985.0,"Adventure, Comedy, Sci-Fi",915281.0,7/3/1985,Robert Zemeckis


In [3]:
#Extract relevant columns
movies = movies[['Const', 'Title', 'Runtime (mins)', 'Year', 'Genres', 'Release Date', 'IMDb Rating']]
movies.head()

,Const,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating
0,tt0110912,Pulp Fiction,154.0,1994.0,"Crime, Drama",5/21/1994,8.9
1,tt1872181,The Amazing Spider-Man 2,142.0,2014.0,"Action, Adventure, Sci-Fi",4/10/2014,6.6
2,tt0111161,The Shawshank Redemption,142.0,1994.0,Drama,9/10/1994,9.3
3,tt0076759,Star Wars,121.0,1977.0,"Action, Adventure, Fantasy, Sci-Fi",5/25/1977,8.6
4,tt0088763,Back to the Future,116.0,1985.0,"Adventure, Comedy, Sci-Fi",7/3/1985,8.5


In [4]:
#Rename Columns
movies = movies.rename(columns= {'Const': 'IMDb ID'})
movies.head()

,IMDb ID,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating
0,tt0110912,Pulp Fiction,154.0,1994.0,"Crime, Drama",5/21/1994,8.9
1,tt1872181,The Amazing Spider-Man 2,142.0,2014.0,"Action, Adventure, Sci-Fi",4/10/2014,6.6
2,tt0111161,The Shawshank Redemption,142.0,1994.0,Drama,9/10/1994,9.3
3,tt0076759,Star Wars,121.0,1977.0,"Action, Adventure, Fantasy, Sci-Fi",5/25/1977,8.6
4,tt0088763,Back to the Future,116.0,1985.0,"Adventure, Comedy, Sci-Fi",7/3/1985,8.5


In [5]:
#Drop rows with incomplete data
movies = movies.dropna()

#Exract rows from years 2010 and later
movies = movies[movies.iloc[:,3] >= 2010]
movies = movies.set_index(['IMDb ID'])
movies.head()

,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating
IMDb ID,,,,,,
tt1872181,The Amazing Spider-Man 2,142.0,2014.0,"Action, Adventure, Sci-Fi",4/10/2014,6.6
tt1323594,Despicable Me,95.0,2010.0,"Animation, Comedy, Family, Fantasy",6/20/2010,7.7
tt1375670,Grown Ups,102.0,2010.0,Comedy,6/24/2010,6.0
tt0892769,How to Train Your Dragon,98.0,2010.0,"Animation, Action, Adventure, Family, Fantasy",3/18/2010,8.1
tt1375666,Inception,148.0,2010.0,"Action, Adventure, Sci-Fi, Thriller",7/8/2010,8.8


In [6]:
#Change Year and Runtime to integers
movies[['Year','Runtime (mins)']] = movies[['Year','Runtime (mins)']].applymap(np.int64)
movies.head()

,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating
IMDb ID,,,,,,
tt1872181,The Amazing Spider-Man 2,142,2014,"Action, Adventure, Sci-Fi",4/10/2014,6.6
tt1323594,Despicable Me,95,2010,"Animation, Comedy, Family, Fantasy",6/20/2010,7.7
tt1375670,Grown Ups,102,2010,Comedy,6/24/2010,6.0
tt0892769,How to Train Your Dragon,98,2010,"Animation, Action, Adventure, Family, Fantasy",3/18/2010,8.1
tt1375666,Inception,148,2010,"Action, Adventure, Sci-Fi, Thriller",7/8/2010,8.8


In [7]:
# Pull: Rating, Metascore, and BoxOffice from omdb
movies['Metascore'] = ''
movies['Rating'] = ''
movies['Box Office'] = ''
count=0

movies.head()

,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating,Metascore,Rating,Box Office
IMDb ID,,,,,,,,,
tt1872181,The Amazing Spider-Man 2,142,2014,"Action, Adventure, Sci-Fi",4/10/2014,6.6,,,
tt1323594,Despicable Me,95,2010,"Animation, Comedy, Family, Fantasy",6/20/2010,7.7,,,
tt1375670,Grown Ups,102,2010,Comedy,6/24/2010,6.0,,,
tt0892769,How to Train Your Dragon,98,2010,"Animation, Action, Adventure, Family, Fantasy",3/18/2010,8.1,,,
tt1375666,Inception,148,2010,"Action, Adventure, Sci-Fi, Thriller",7/8/2010,8.8,,,


In [8]:
for imdb in movies.index:
    try:
        count += 1
        url = 'http://www.omdbapi.com/?i={0}&apikey=trilogy'.format(imdb)
        results = requests.get(url).json()
        movies.loc[imdb,'Rating'] = results['Rated']
        movies.loc[imdb,'Metascore'] = results['Metascore']
        movies.loc[imdb,'Box Office'] = results['BoxOffice']
        print(f'{imdb} processed: {count} / {len(movies.index)}')
        time.sleep(.1)
    except:
        print(f'Error processing {imdb}, skipping: {count} / {len(movies.index)}')

tt1872181 processed: 1 / 1562
tt1323594 processed: 2 / 1562
tt1375670 processed: 3 / 1562
tt0892769 processed: 4 / 1562
tt1375666 processed: 5 / 1562
tt1228705 processed: 6 / 1562
tt1250777 processed: 7 / 1562
tt1245526 processed: 8 / 1562
tt1130884 processed: 9 / 1562
tt1104001 processed: 10 / 1562
tt0435761 processed: 11 / 1562
tt0780504 processed: 12 / 1562
tt1596343 processed: 13 / 1562
tt1201607 processed: 14 / 1562
tt1219289 processed: 15 / 1562
tt1298650 processed: 16 / 1562
tt1270798 processed: 17 / 1562
tt1568346 processed: 18 / 1562
tt1454029 processed: 19 / 1562
tt1637688 processed: 20 / 1562
tt1232829 processed: 21 / 1562
tt0948470 processed: 22 / 1562
tt1605630 processed: 23 / 1562
tt0848228 processed: 24 / 1562
tt1440129 processed: 25 / 1562
tt1194173 processed: 26 / 1562
tt1217209 processed: 27 / 1562
tt1259521 processed: 28 / 1562
tt1790886 processed: 29 / 1562
tt1371111 processed: 30 / 1562
tt1345836 processed: 31 / 1562
tt1077368 processed: 32 / 1562
tt1853728 process

tt1401152 processed: 262 / 1562
tt1240982 processed: 263 / 1562
tt1657507 processed: 264 / 1562
tt0477302 processed: 265 / 1562
tt1448755 processed: 266 / 1562
tt1204342 processed: 267 / 1562
tt0448694 processed: 268 / 1562
tt0471042 processed: 269 / 1562
tt1568911 processed: 270 / 1562
tt1591479 processed: 271 / 1562
tt1366365 processed: 272 / 1562
tt1142977 processed: 273 / 1562
tt1838544 processed: 274 / 1562
tt1397514 processed: 275 / 1562
tt1327194 processed: 276 / 1562
tt1667353 processed: 277 / 1562
tt2109184 processed: 278 / 1562
tt0431021 processed: 279 / 1562
tt1899353 processed: 280 / 1562
tt2083383 processed: 281 / 1562
tt1496025 processed: 282 / 1562
tt1596365 processed: 283 / 1562
tt1959332 processed: 284 / 1562
tt1213663 processed: 285 / 1562
tt1645155 processed: 286 / 1562
tt1132449 processed: 287 / 1562
tt1990314 processed: 288 / 1562
tt1999995 processed: 289 / 1562
tt1308729 processed: 290 / 1562
tt0404978 processed: 291 / 1562
tt1748179 processed: 292 / 1562
tt176418

tt1606392 processed: 520 / 1562
tt1723124 processed: 521 / 1562
tt1441326 processed: 522 / 1562
tt1440161 processed: 523 / 1562
tt0997152 processed: 524 / 1562
tt1464580 processed: 525 / 1562
tt1529572 processed: 526 / 1562
tt1423995 processed: 527 / 1562
tt1440292 processed: 528 / 1562
tt1742334 processed: 529 / 1562
tt2771372 processed: 530 / 1562
tt1273678 processed: 531 / 1562
tt0808510 processed: 532 / 1562
tt1666186 processed: 533 / 1562
tt1032751 processed: 534 / 1562
tt1464174 processed: 535 / 1562
tt1604171 processed: 536 / 1562
tt1449283 processed: 537 / 1562
tt1767382 processed: 538 / 1562
tt1630036 processed: 539 / 1562
tt1517489 processed: 540 / 1562
tt1870529 processed: 541 / 1562
tt1714206 processed: 542 / 1562
tt1160996 processed: 543 / 1562
tt0455323 processed: 544 / 1562
tt1990216 processed: 545 / 1562
tt1438173 processed: 546 / 1562
tt2061712 processed: 547 / 1562
tt1867093 processed: 548 / 1562
tt2119474 processed: 549 / 1562
tt1886493 processed: 550 / 1562
tt164978

tt1730687 processed: 777 / 1562
tt1833879 processed: 778 / 1562
tt1740047 processed: 779 / 1562
tt1386925 processed: 780 / 1562
tt1196340 processed: 781 / 1562
tt1390535 processed: 782 / 1562
tt1381505 processed: 783 / 1562
tt1518812 processed: 784 / 1562
tt1409004 processed: 785 / 1562
tt1130969 processed: 786 / 1562
tt1106860 processed: 787 / 1562
tt1726592 processed: 788 / 1562
tt1937388 processed: 789 / 1562
tt1709652 processed: 790 / 1562
tt1838722 processed: 791 / 1562
tt1414378 processed: 792 / 1562
tt1575539 processed: 793 / 1562
tt1125929 processed: 794 / 1562
tt1092634 processed: 795 / 1562
tt1278379 processed: 796 / 1562
tt1411232 processed: 797 / 1562
tt1600524 processed: 798 / 1562
tt1325723 processed: 799 / 1562
tt1233192 processed: 800 / 1562
tt2112277 processed: 801 / 1562
tt2062661 processed: 802 / 1562
tt1300159 processed: 803 / 1562
tt1563704 processed: 804 / 1562
tt1038685 processed: 805 / 1562
tt1407049 processed: 806 / 1562
tt1196134 processed: 807 / 1562
tt249510

tt1462054 processed: 1033 / 1562
tt1758575 processed: 1034 / 1562
tt1555149 processed: 1035 / 1562
tt2011971 processed: 1036 / 1562
tt0850677 processed: 1037 / 1562
tt1447793 processed: 1038 / 1562
tt1613062 processed: 1039 / 1562
tt1728196 processed: 1040 / 1562
tt1646974 processed: 1041 / 1562
tt1278449 processed: 1042 / 1562
tt1650453 processed: 1043 / 1562
tt1303803 processed: 1044 / 1562
tt1535491 processed: 1045 / 1562
tt1583753 processed: 1046 / 1562
tt1692500 processed: 1047 / 1562
tt2521086 processed: 1048 / 1562
tt1830497 processed: 1049 / 1562
tt1303235 processed: 1050 / 1562
tt2009538 processed: 1051 / 1562
tt1285016 processed: 1052 / 1562
tt1675434 processed: 1053 / 1562
tt0975645 processed: 1054 / 1562
tt1210166 processed: 1055 / 1562
tt1655420 processed: 1056 / 1562
tt1979320 processed: 1057 / 1562
tt1542344 processed: 1058 / 1562
tt0964517 processed: 1059 / 1562
tt0993846 processed: 1060 / 1562
tt1426329 processed: 1061 / 1562
tt1571222 processed: 1062 / 1562
tt1291580 

tt2374835 processed: 1279 / 1562
tt1717578 processed: 1280 / 1562
tt1747960 processed: 1281 / 1562
tt1087470 processed: 1282 / 1562
tt1591585 processed: 1283 / 1562
tt2426110 processed: 1284 / 1562
tt1632679 processed: 1285 / 1562
tt1719681 processed: 1286 / 1562
tt1844811 processed: 1287 / 1562
tt2345525 processed: 1288 / 1562
tt1684934 processed: 1289 / 1562
tt1705115 processed: 1290 / 1562
tt2137742 processed: 1291 / 1562
tt1595411 processed: 1292 / 1562
tt1831611 processed: 1293 / 1562
tt1685518 processed: 1294 / 1562
tt2302925 processed: 1295 / 1562
tt2246779 processed: 1296 / 1562
tt1844641 processed: 1297 / 1562
Error processing tt0944947, skipping: 1298 / 1562
tt1392190 processed: 1299 / 1562
tt1945228 processed: 1300 / 1562
tt0478970 processed: 1301 / 1562
tt1289401 processed: 1302 / 1562
tt1825683 processed: 1303 / 1562
tt2094766 processed: 1304 / 1562
tt1340138 processed: 1305 / 1562
tt1029360 processed: 1306 / 1562
tt1964418 processed: 1307 / 1562
tt1663202 processed: 1308 

tt2017486 processed: 1527 / 1562
tt1837636 processed: 1528 / 1562
tt2179136 processed: 1529 / 1562
tt1355644 processed: 1530 / 1562
tt1881002 processed: 1531 / 1562
tt1469304 processed: 1532 / 1562
tt2402927 processed: 1533 / 1562
tt2374684 processed: 1534 / 1562
tt1703957 processed: 1535 / 1562
tt2494376 processed: 1536 / 1562
Error processing tt2879552, skipping: 1537 / 1562
tt2062700 processed: 1538 / 1562
tt2671706 processed: 1539 / 1562
tt2788710 processed: 1540 / 1562
tt1754656 processed: 1541 / 1562
tt2802144 processed: 1542 / 1562
tt2397535 processed: 1543 / 1562
tt3353060 processed: 1544 / 1562
tt2884018 processed: 1545 / 1562
tt2937898 processed: 1546 / 1562
tt3397884 processed: 1547 / 1562
tt2180411 processed: 1548 / 1562
tt0790770 processed: 1549 / 1562
tt3829266 processed: 1550 / 1562
tt2494362 processed: 1551 / 1562
tt3289728 processed: 1552 / 1562
tt2726552 processed: 1553 / 1562
tt1658801 processed: 1554 / 1562
tt3297330 processed: 1555 / 1562
tt3707106 processed: 1556 

In [9]:
movies

,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating,Metascore,Rating,Box Office
IMDb ID,,,,,,,,,
tt1872181,The Amazing Spider-Man 2,142,2014,"Action, Adventure, Sci-Fi",4/10/2014,6.6,53,PG-13,"$183,277,573"
tt1323594,Despicable Me,95,2010,"Animation, Comedy, Family, Fantasy",6/20/2010,7.7,72,PG,"$251,476,985"
tt1375670,Grown Ups,102,2010,Comedy,6/24/2010,6.0,30,PG-13,"$162,001,186"
tt0892769,How to Train Your Dragon,98,2010,"Animation, Action, Adventure, Family, Fantasy",3/18/2010,8.1,74,PG,"$216,900,000"
tt1375666,Inception,148,2010,"Action, Adventure, Sci-Fi, Thriller",7/8/2010,8.8,74,PG-13,"$292,568,851"
tt1228705,Iron Man 2,124,2010,"Action, Adventure, Sci-Fi",4/26/2010,7.0,57,PG-13,"$312,057,433"
tt1250777,Kick-Ass,117,2010,"Action, Comedy",3/12/2010,7.6,66,R,"$20,000,000"
tt1245526,RED,111,2010,"Action, Comedy, Crime, Thriller",9/29/2010,7.1,60,PG-13,"$88,900,000"
tt1130884,Shutter Island,138,2010,"Mystery, Thriller",2/13/2010,8.1,63,R,"$125,001,000"


In [10]:
#Clean df
movies = movies.dropna()
movies = movies[~movies['Box Office'].str.contains("N/A")]
movies = movies[~movies['Rating'].str.contains("N/A")]
movies = movies[~movies['Metascore'].str.contains("N/A")]

movies

,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating,Metascore,Rating,Box Office
IMDb ID,,,,,,,,,
tt1872181,The Amazing Spider-Man 2,142,2014,"Action, Adventure, Sci-Fi",4/10/2014,6.6,53,PG-13,"$183,277,573"
tt1323594,Despicable Me,95,2010,"Animation, Comedy, Family, Fantasy",6/20/2010,7.7,72,PG,"$251,476,985"
tt1375670,Grown Ups,102,2010,Comedy,6/24/2010,6.0,30,PG-13,"$162,001,186"
tt0892769,How to Train Your Dragon,98,2010,"Animation, Action, Adventure, Family, Fantasy",3/18/2010,8.1,74,PG,"$216,900,000"
tt1375666,Inception,148,2010,"Action, Adventure, Sci-Fi, Thriller",7/8/2010,8.8,74,PG-13,"$292,568,851"
tt1228705,Iron Man 2,124,2010,"Action, Adventure, Sci-Fi",4/26/2010,7.0,57,PG-13,"$312,057,433"
tt1250777,Kick-Ass,117,2010,"Action, Comedy",3/12/2010,7.6,66,R,"$20,000,000"
tt1245526,RED,111,2010,"Action, Comedy, Crime, Thriller",9/29/2010,7.1,60,PG-13,"$88,900,000"
tt1130884,Shutter Island,138,2010,"Mystery, Thriller",2/13/2010,8.1,63,R,"$125,001,000"


In [12]:
#Convert Box Office column to int
movies['Box Office'] = movies[movies.columns[8:]].replace('[\$,]', '', regex=True).astype(float)
movies['Box Office'] = movies['Box Office'].astype(int)
movies

,Title,Runtime (mins),Year,Genres,Release Date,IMDb Rating,Metascore,Rating,Box Office
IMDb ID,,,,,,,,,
tt1872181,The Amazing Spider-Man 2,142,2014,"Action, Adventure, Sci-Fi",4/10/2014,6.6,53,PG-13,183277573
tt1323594,Despicable Me,95,2010,"Animation, Comedy, Family, Fantasy",6/20/2010,7.7,72,PG,251476985
tt1375670,Grown Ups,102,2010,Comedy,6/24/2010,6.0,30,PG-13,162001186
tt0892769,How to Train Your Dragon,98,2010,"Animation, Action, Adventure, Family, Fantasy",3/18/2010,8.1,74,PG,216900000
tt1375666,Inception,148,2010,"Action, Adventure, Sci-Fi, Thriller",7/8/2010,8.8,74,PG-13,292568851
tt1228705,Iron Man 2,124,2010,"Action, Adventure, Sci-Fi",4/26/2010,7.0,57,PG-13,312057433
tt1250777,Kick-Ass,117,2010,"Action, Comedy",3/12/2010,7.6,66,R,20000000
tt1245526,RED,111,2010,"Action, Comedy, Crime, Thriller",9/29/2010,7.1,60,PG-13,88900000
tt1130884,Shutter Island,138,2010,"Mystery, Thriller",2/13/2010,8.1,63,R,125001000


In [13]:
# Save Data to csv
movies.to_csv("Movie_Data.csv")

In [ ]:
#Critics Ratings

#subquery

#Scatterplots to visualize correlation

#test for significance 

In [ ]:
#Genres

#Paredo bar Chart to identify top Genres

#Plot changes over time with a Line graph

In [ ]:
#MPAA Rating

#Paredo bar Chart

In [ ]:
#Runtime

#Scatterplot

#test for significance 